In [ ]:
import pandas as pd
import os
import flask

In [8]:
#I got this from google, but just ensuring the file is not corrupted and will open

df = pd.read_csv('Transactions.csv')

print(df.to_string())

         Date           Description  Amount      Type        Category
0    4/9/2025                 JACKS   29.00  Purchase  Food/Groceries
1    4/8/2025  WAL-MART SUPERCENTER    8.78  Purchase   Miscellaneous
2    4/5/2025          AMC THEATERS   35.78  Purchase   Entertainment
3    4/3/2025     PAYPAL *CHEWY INC   77.92  Purchase   Miscellaneous
4    4/1/2025             BRAVO #16   25.17  Purchase  Transportation
5    4/1/2025  WAL-MART SUPERCENTER   10.75  Purchase   Miscellaneous
6   3/28/2025  WAL-MART SUPERCENTER   27.07  Purchase   Miscellaneous
7   3/22/2025                SUNOCO   25.98  Purchase  Transportation
8   3/12/2025               TJ MAXX   61.76  Purchase   Miscellaneous
9   3/12/2025             PAPAJOHNS   16.49  Purchase  Food/Groceries
10  3/12/2025     PAYPAL *FOREVER21   73.22  Purchase   Miscellaneous
11  3/12/2025               SEPHORA   88.41  Purchase   Miscellaneous
12  3/12/2025                  ALDI  101.76  Purchase  Food/Groceries
13   3/9/2025       

In [10]:
CSV_FILE = 'Transactions.csv'
INITIAL_BALANCE = 100.0

class AccountManager:
    def __init__(self,initial_balance = INITIAL_BALANCE,csv_file = CSV_FILE):
            self.csv_file = csv_file
            self.initial_balance = initial_balance
            self.df = self.load_file()


    def load_file(self):
            """Loads data from the CSV file"""
            if os.path.exists(self.csv_file):
                try:
                    df = pd.read_csv(self.csv_file)
                    print(f"File exists as {self.csv_file}")
                    return df
                except pd.errors.EmptyDataError:        #I did look this except up on google but it seems to do what I'm wanting
                    print("Error: The file is empty.")
                    return None
            else:
                print(f"{self.csv_file} File not found")
                return None
            
    def __str__(self):
         s = (f"Database: {self.csv_file}, Initial Balance: {self.initial_balance}\n")
         if self.df is not None:
              s += ("DataFrame contents:\n")
              s += self.df.to_string()
              return s

am = AccountManager('Transactions.csv')

File exists as Transactions.csv


In [11]:
print(am)

Database: Transactions.csv, Initial Balance: Transactions.csv
DataFrame contents:
         Date           Description  Amount      Type        Category
0    4/9/2025                 JACKS   29.00  Purchase  Food/Groceries
1    4/8/2025  WAL-MART SUPERCENTER    8.78  Purchase   Miscellaneous
2    4/5/2025          AMC THEATERS   35.78  Purchase   Entertainment
3    4/3/2025     PAYPAL *CHEWY INC   77.92  Purchase   Miscellaneous
4    4/1/2025             BRAVO #16   25.17  Purchase  Transportation
5    4/1/2025  WAL-MART SUPERCENTER   10.75  Purchase   Miscellaneous
6   3/28/2025  WAL-MART SUPERCENTER   27.07  Purchase   Miscellaneous
7   3/22/2025                SUNOCO   25.98  Purchase  Transportation
8   3/12/2025               TJ MAXX   61.76  Purchase   Miscellaneous
9   3/12/2025             PAPAJOHNS   16.49  Purchase  Food/Groceries
10  3/12/2025     PAYPAL *FOREVER21   73.22  Purchase   Miscellaneous
11  3/12/2025               SEPHORA   88.41  Purchase   Miscellaneous
12  3/12

In [12]:
import csv
import os
import pandas as pd

#Copied and edited from HCI 574 lecture 36
from flask import Flask, render_template, request, redirect, url_for

In [ ]:
app = Flask(__name__)

@app.route("/")  
def index():
    html_str = render_template('index.html', title="Landing Page") # title will be inlined in {{ title }}
    print(html_str) # DEBUG
    return html_str  # give it to the browser to display the inline page

app.run(debug=False, port=8080) 



CSV_FILE = 'Transactions.csv'
INITIAL_BALANCE = 100.0


class AccountManager:
    def __init__(self,initial_balance = INITIAL_BALANCE,csv_file = CSV_FILE):
            self.csv_file = csv_file
            self.initial_balance = initial_balance
            self.activities = self._load_activities()
            self.df = self.load_file()


    def load_file(self):
            """Loads data from the CSV file"""
            if os.path.exists(self.csv_file):
                try:
                    df = pd.read_csv(self.csv_file)
                    print(f"File exists as {self.csv_file}")
                    return df
                except pd.errors.EmptyDataError:        #I did look this except up on google but it seems to do what I'm wanting
                    print("Error: The file is empty.")
                    return None
            else:
                print(f"{self.csv_file} File not found")
                return None
            
    def __str__(self):
         s = (f"Database: {self.csv_file}, Initial Balance: {self.initial_balance}\n")
         if self.df is not None:
              s += ("DataFrame contents:\n")
              s += self.df.to_string()
              return s


    def save(self):
        """Saves the current data and activity"""
        #Not sure if this is right???
        self.activities.to_csv(self.csv_file, index=False)

    def update_balance(self):
        """Updates the current balance based on user activity, 
        after initial balance. Should update after loading"""
        current_balance = self.initial_balance
        #figure out how add the purhcase amount rows from CSV file
        
    @app.route('/add_transaction', methods=['POST'])
    def add_activity(self, type, catagory, amount):
        """Adds new activity and will sort what type of activity it is(refund or charge),
        catagory(grocery/transportation), and the amount."""
        if request.method =='POST':
            transaction_type = request.form['transaction_type']
            store = request.form['store']
            amount = request.form['amount']
            date = request.form['date']

            #Append data to CSV file
            #I got this from the internet
            try:
                with open(CSV_FILE, 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([transaction_type, store, amount, date])

                return render_template('index.html', message="Transaction added successfully!", message_type="success")
            except Exception as e:
                return render_template('index.html', message=f"Error adding transaction: {e}", message_type="error")

    def purchase(self, amount, catagory):
        """Adds purchase activity ans subtracts from the balance. Hopefully warns & 
        denies the purchase if the user doesn't have the balance to cover it,"""


    def get_balance(self):
        """Gives user their current balance."""

    def get_activity(self):
        """Gives user their activity."""

    def plot(self):
        """Plots the balance over time."""
        #Bar chart or graph.....lookup what is most popular

#need a def for handling str + floats for consistency
#maybe defs for UI(menu, tables, charts, viewing activity)
def main():
    """Runs the Account Manager for users to input data."""

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit


127.0.0.1 - - [07/Jul/2025 16:22:59] "GET / HTTP/1.1" 200 -


<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Landing Page</title>
</head>
<body>
    <div>
        <h1>Welcome!</h1>
         <p>This is a simple Flask application test.</p>
    </div>
    <div>
        <h2>Add a New Transaction</h2>
        <form action="/add_transaction" method="post">
            <label for="transaction_type">Transaction Type:</label>
            <select id="transaction_type" name="transaction_type" required>
                <option value="Purchase">Purchase</option>
                <option value="Refund">Refund</option>
            </select>

            <label for="store">Buisness Establishment:</label>
            <input type="text" id="Buisness Establishment" name="Buisness Establishment" required>

            <label for="amount">Amount:</label>
            <input type="number" id="amount" name="amount" step="0.01" required>

            <label for="date"